In [3]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

## Model

We build a class to handle API calls to the LLM, in case we decide to change the LLM model.

Please make sure you have obtained the appropriate API key before continuing.

In [23]:
class AbstractModel:
    def __init__(self, base: str):
        pass

    def query(self, prompt: str) -> str:
        raise NotImplementedError

class QueryException(Exception):
    def __init__(self, message: str):
        super().__init__(f"Failed to query to the LLM: {message}")

In [24]:
class GeminiModel(AbstractModel):
    def __init__(self, base: str):
        self.base = base
        self.client = genai.GenerativeModel('gemini-1.5-pro')

    def query(self, prompt: str) -> str:
        response = self.client.generate_content(self.base.format(prompt=prompt)).candidates[0]
        if not hasattr(response, 'content'):
            raise QueryException('LLM response does not have content')
        return response.content.parts[0].text

## Prompt

We design the prompts and attempt to get the answer here.

In [9]:
BASE_PROMPT = """
{prompt}
"""

In [25]:
gemini_model = GeminiModel(BASE_PROMPT)

In [26]:
gemini_model.query("Hello, how are you today?")

"As an AI, I don't have feelings, but thanks for asking! I am ready to assist you with whatever you need. How can I help you today? 😊 \n"